In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import os

In [0]:
os.chdir('./drive/My Drive/drive/舆情监控之情感分析')

In [0]:
os.listdir('./')

['情感分析.ipynb',
 '情感分析1.ipynb',
 'ratings_train.csv',
 'ratings_valid.csv',
 'qingganfenxi-model.pt',
 'stoi.npy',
 '情感分析模型加载.ipynb',
 '情感分析2.ipynb']

In [0]:
import torchtext.data as data
import torch

In [0]:
def tokenize(text):
  return [item for item in str(text)]

In [0]:
TEXT = data.Field(tokenize=tokenize)
LABEL = data.LabelField(dtype=torch.float)

In [0]:
fields = [('rating',LABEL),('comment',TEXT)]
train, val = data.TabularDataset.splits(
    path='./', train='ratings_train.csv', validation='ratings_valid.csv', format='csv',
    skip_header=True, fields=fields,
)

In [0]:
print(vars(val.examples[0]))

{'rating': '0.0', 'comment': ['B', 'F', '家', '3', '个', '寿', '星', '一', '起', '过', '生', '日', '，', '在', '这', '吃', '的', '。', '\n', '\n', '\n', '前', '一', '天', '被', '告', '知', '在', '三', '里', '河', '附', '近', '的', '大', '连', '海', '鲜', '吃', '，', '我', '就', '上', '点', '评', '查', '了', '一', '下', '，', '看', '描', '述', '应', '该', '是', '这', '家', '店', '了', '，', '竟', '然', '只', '有', '1', '颗', '星', '-', '。', '-', '！', '不', '知', '道', '是', '谁', '选', '的', '这', '里', '呢', '…', '…', '\n', '\n', '\n', '也', '许', '是', '做', '足', '了', '心', '理', '准', '备', '，', '所', '以', '觉', '得', '味', '道', '也', '没', '差', '到', '1', '颗', '星', '的', '地', '步', '，', '姑', '且', '给', '个', '2', '颗', '星', '吧', '…', '…', '\n', '\n', '\n', '菜', '的', '口', '味', '普', '遍', '欠', '佳', '，', '卖', '相', '也', '不', '是', '很', '好', '，', '印', '象', '最', '深', '的', '是', '扇', '贝', '，', '小', '的', '可', '怜', '，', '真', '的', '很', '可', '怜', '，', '加', '上', '豆', '豉', '，', '那', '个', '贝', '也', '就', '跟', '1', '毛', '钱', '硬', '币', '那', '么', '大', '，', '愧', '对', '了', '扇', '贝', '这', '个', '

In [0]:
TEXT.build_vocab(train)
LABEL.build_vocab(train)

In [0]:
print(TEXT.vocab.freqs.most_common(10))

[('，', 2550171), ('\n', 2029596), ('的', 1712670), ('。', 1024074), ('不', 894025), ('是', 890411), ('了', 834490), ('一', 694260), ('有', 485314), ('吃', 462237)]


In [0]:
TEXT.vocab.stoi['我']

13

In [0]:
import numpy as np

In [0]:
print(TEXT.vocab.stoi)
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7fc5cec13d90>, {'<unk>': 0, '<pad>': 1, '，': 2, '\n': 3, '的': 4, '。': 5, '不': 6, '是': 7, '了': 8, '一': 9, '有': 10, '吃': 11, '很': 12, '我': 13, '好': 14, '个': 15, ' ': 16, '还': 17, '就': 18, '这': 19, '也': 20, '点': 21, '去': 22, '来': 23, '！': 24, '人': 25, '都': 26, '没': 27, '上': 28, '家': 29, '在': 30, '味': 31, '\xa0': 32, '菜': 33, '~': 34, '多': 35, '道': 36, '说': 37, '以': 38, '么': 39, '过': 40, '大': 41, '.': 42, '里': 43, '要': 44, '到': 45, '面': 46, '服': 47, '们': 48, '可': 49, '小': 50, '务': 51, '店': 52, '那': 53, '后': 54, '时': 55, '得': 56, '子': 57, '看': 58, '次': 59, '下': 60, '错': 61, '和': 62, ',': 63, '会': 64, '觉': 65, '他': 66, '比': 67, '肉': 68, '但': 69, '真': 70, '然': 71, '员': 72, '饭': 73, '0': 74, '感': 75, '口': 76, '价': 77, '1': 78, '天': 79, '能': 80, '最': 81, '地': 82, '为': 83, '什': 84, '太': 85, '啊': 86, '餐': 87, '想': 88, '差': 89, '样': 90, '鱼': 91, '果': 92, '老': 93, '2': 94, '只': 95, '着': 96, '实': 97, '开': 98, '再': 99, '生': 100, '西': 101, '-': 102, '常': 1

In [0]:
batch_size = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
train_iterator, val_iterator = data.BucketIterator.splits(
    (train, val),
    batch_size = batch_size,
    device = device,
    sort_key = lambda x: len(x.comment),
    sort = False,
)

In [0]:
import torch.nn as nn
import torch

class RNN(nn.Module):
  
  def __init__(self, vocab_size, embedding_dim, padding_idx, hidden_dim, num_layers, bidirectional, dropout, output_dim):
    super(RNN,self).__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=padding_idx)
    self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, bidirectional=bidirectional, dropout=dropout)
    self.dropout = nn.Dropout(dropout)
    self.decoder = nn.Linear(hidden_dim*2, output_dim)
    self.hidden_dim = hidden_dim
    self.num_layers = num_layers
    self.output_dim = output_dim

  def forward(self, text, hidden):
    #embedded=[sentence_len,batch_size,embedding_dim]
    embedded = self.dropout(self.embedding(text))
    #output=[sentence_len,batch_size,hidden_dim*num_directional]
    #hidden=[num_layers*num_directional,batch_size,hidden_dim] 
    #cell=[num_layers*num_directional,batch_size,hidden_dim] 
    output, (hidden, cell) = self.rnn(embedded, hidden)
    decoded = self.decoder(self.dropout(torch.cat((hidden[-2,:,:],hidden[-1,:,:]),dim=1)))
    return decoded, (hidden, cell)

  def init_hidden(self, batch_size, requires_grad=True):
    weight = next(self.parameters())
    return (
        weight.new_zeros((self.num_layers*2, batch_size, hidden_dim),requires_grad=requires_grad),
        weight.new_zeros((self.num_layers*2, batch_size, hidden_dim),requires_grad=requires_grad)
    )

In [0]:
vocab_size = len(TEXT.vocab)
embedding_dim = 100 
padding_idx = TEXT.vocab.stoi[TEXT.pad_token]
hidden_dim = 128
num_layers = 2
bidirectional = True
dropout = 0.5
output_dim = 2

In [0]:
type(TEXT.vocab.stoi)

collections.defaultdict

In [0]:
import numpy as np

In [0]:
np.save('stoi',TEXT.vocab.stoi)

In [0]:
model = RNN(vocab_size, embedding_dim, padding_idx, hidden_dim, num_layers, bidirectional, dropout, output_dim)

In [0]:
model.embedding.weight.data.shape

torch.Size([8167, 100])

In [0]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(embedding_dim)
model.embedding.weight.data[PAD_IDX] = torch.zeros(embedding_dim)

In [0]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()
# loss_fn = nn.BCEWithLogitsLoss()
model = model.to(device)
loss_fn = loss_fn.to(device)

In [0]:
# Remove this part
def repackage_hidden(h):
    """Wraps hidden states in new Tensors, to detach them from their history."""
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)

In [0]:
def train(model, iterator, optimizer, loss_fn):
  epoch_loss = 0
  model.train()
  hidden = model.init_hidden(batch_size)

  for i,batch in enumerate(iterator):
    if batch.comment.shape[1] == 64:
      optimizer.zero_grad()
      hidden = repackage_hidden(hidden)
      predictions, hidden = model(batch.comment, hidden)
      predictions = predictions.squeeze(1)
      # loss = loss_fn(predictions, batch.rating)
      loss = loss_fn(predictions, batch.rating.long())
      loss.backward()
      optimizer.step()
      epoch_loss += loss.item()
      if i % 1000 == 0:
        print(loss.item())
    else:
      continue

  return epoch_loss / len(iterator)

In [0]:
def evaluate(model, iterator, loss_fn):
  epoch_loss = 0
  model.eval()
  hidden = model.init_hidden(batch_size,requires_grad=False)

  with torch.no_grad():
    for batch in iterator:
      if batch.comment.shape[1] == 64:
        hidden = repackage_hidden(hidden)
        predictions, hidden = model(batch.comment, hidden)
        predictions = predictions.squeeze(1)
        # loss = loss_fn(predictions, batch.rating)
        loss = loss_fn(predictions, batch.rating.long())
        epoch_loss += loss.item()
      else:
        continue

  return epoch_loss / len(iterator)

In [0]:
EPOCHS = 1
best_valid_loss = float('inf')
for epoch in range(EPOCHS):
  train_loss = train(model, train_iterator, optimizer, loss_fn)
  valid_loss = evaluate(model, val_iterator, loss_fn)

  if valid_loss < best_valid_loss:
    best_valid_loss = valid_loss
    torch.save(model.state_dict(),'./qingganfenxi-model.pt')
  
  print('****************')
  print(train_loss)
  print(valid_loss)
  print('****************')

0.6936320066452026
0.3189755380153656
0.3076046109199524
0.470416396856308
0.24056550860404968
****************
0.3457575502838407
0.2031665003846089
****************


In [0]:
model.embedding.weight.data.shape

torch.Size([6708, 100])

In [0]:
import torch.nn.functional as F
device = torch.device('cuda')
# model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load('./qingganfenxi-model.pt'))
model.to(device)
def predict_sentiment(sentence):
    hidden = model.init_hidden(1)
    tokenized = [tok for tok in sentence]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    output, hidden = model(tensor, hidden)
    output = output[0]
    
    prediction = F.softmax(output,dim=0)
    return prediction
    # return output

In [0]:
predict_sentiment('不错的一家餐厅，在香格里拉的一楼。\r\r\n座位比较少，很安静。\r\r\n食物也蛮新鲜的。\r\r\n或许和就餐时的心情有关，东西也觉得很棒。\r\r\n总的来说不错啦，有机会还会去的。\r\r\n')

tensor([0.9965, 0.0035], device='cuda:0', grad_fn=<SoftmaxBackward>)

In [0]:
predict_sentiment('接电话的小姐态度很不好，我问他又没有教小孩的初级班，没有你就说没有，她说4个人可以开班，你自己找人去。我说你办培训班，我去找人，我上哪儿去找人?实在是太搞笑了。不知道老板是怎么想的，找这样的人来接电话，一天不知道给他得罪多少客人，连接电话的基本规矩都不懂，好吧，本来还想带孩子去学滑冰的，现在肯定不会再在这里学了，而且我也会在我的圈子里说不好的了。\r\r\n')

tensor([0.0514, 0.9486], device='cuda:0', grad_fn=<SoftmaxBackward>)

In [0]:
predict_sentiment('初时认识这家店，以为是必胜客的山寨版，并未多加留意。\r\r\n直到在嘉茂吃过一次披萨，加上一喝难忘的“缤纷果粒茶”。\r\r\n这不馋意面了，就又自动自觉的来了这家店。\r\r\n缤纷果粒茶，已经由10元涨到了12元，而且里料比之前少了。但得认同的是现在什么都在涨价。\r\r\n特色香烤鸡翼，16元，四个，很肥美，比神马KFC啊，必胜客的好吃多了。\r\r\n肉酱局饭，14元，肉酱和奶酪分别在盘子的两边，吃时混合，奶酷很香浓，赞！\r\r\n意大利海鲜面，里面有鱿鱼，虾仁，蛤蜊，虾仁只有一二颗，但比较大，鱿鱼多些，蛤蜊个头不大，但是挺新鲜而且没有沙子。\r\r\n两份主食，一份烤品+一壶茶，共计消费58元，比什么必胜客好吃多多了，而且便宜一大截！\r\r\n')

tensor([0.8267, 0.1733], device='cuda:0', grad_fn=<SoftmaxBackward>)